Inspired by the [tutorial](https://keras.io/examples/lstm_seq2seq/), I am going to implement character-level seq2seq for paraphrasing.

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
from sklearn.model_selection import train_test_split

In [23]:
batch_size = 4  # Batch size for training.
epochs = 1  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
# num_samples = 10000  # Number of samples to train on.

In [56]:
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
        return lines

In [24]:
data_path = 'opusparcus_v2/en-train-100K.txt'
lines = load_data(data_path)

In [25]:
train_lines, val_lines = train_test_split(lines, test_size=0.20, random_state=42)

In [26]:
def parse_input_and_target_from_lines(lines, calc_characters = False):
    input_texts = []
    target_texts = []
    input_characters = set()
    target_characters = set()
    
    for line in train_lines: #lines[: min(num_samples, len(lines) - 1)]:
        splited_lines = line.split('\t')
        input_text = splited_lines[1]
        target_text = splited_lines[2]
        if len(splited_lines) < 3:
            continue

        # input_text, target_text, _ = line.split('\t')
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = '\t' + target_text + '\n'
        input_texts.append(input_text)
        target_texts.append(target_text)
        if calc_characters:
            for char in input_text:
                if char not in input_characters:
                    input_characters.add(char)
            for char in target_text:
                if char not in target_characters:
                    target_characters.add(char)
                    
        
    if calc_characters:
        return input_texts, target_texts, input_characters, target_characters
        
    return input_texts, target_texts


In [27]:
input_texts, target_texts, input_characters, target_characters = parse_input_and_target_from_lines(train_lines, True)

In [28]:
val_input_texts, val_target_texts = parse_input_and_target_from_lines(val_lines)

In [29]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [30]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 80000
Number of unique input tokens: 104
Number of unique output tokens: 142
Max sequence length for inputs: 209
Max sequence length for outputs: 210


In [31]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

As the training data is big, and creating sparse matrix cannot be load into RAM, let's create Dataset Generator to avoid out of memory error.

https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c

In [32]:
from dataset_generator import DataGenerator


In [33]:
dataset = DataGenerator(input_texts=input_texts, target_texts=target_texts,
                        input_token_index=input_token_index, target_token_index=target_token_index,
                        max_encoder_seq_length=max_encoder_seq_length, num_encoder_tokens=num_encoder_tokens,
                        max_decoder_seq_length=max_decoder_seq_length, num_decoder_tokens=num_decoder_tokens,
                        batch_size=batch_size)

In [34]:
val_dataset = DataGenerator(input_texts=val_input_texts, target_texts=val_target_texts,
                        input_token_index=input_token_index, target_token_index=target_token_index,
                        max_encoder_seq_length=max_encoder_seq_length, num_encoder_tokens=num_encoder_tokens,
                        max_decoder_seq_length=max_decoder_seq_length, num_decoder_tokens=num_decoder_tokens,
                        batch_size=batch_size)

In [35]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [36]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [37]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])


In [38]:
model.fit(dataset,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=val_dataset)
# Save model
model.save('s2s_paraphrase.h5')

20000/20000 [==============================] - 813s 41ms/step - loss: 0.1394 - accuracy: 0.9603 - val_loss: 0.1057 - val_accuracy: 0.9689


#### Run several examples to see the results

In [39]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [40]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [70]:
for seq_index in range(1):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = val_dataset.create_encoder_input_item_for_text(val_input_texts[seq_index])
    input_seq = np.array([input_seq])
#     print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', val_input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


-
Input sentence: No , I 'm not kidding you .
Decoded sentence: I 'm gonna take a look .



### Metric

In [71]:
import rouge

In [72]:
dev_data_path = 'opusparcus_v2/en-dev.txt'
dev_lines = load_data(data_path)

In [73]:
dev_input_texts, dev_target_texts = parse_input_and_target_from_lines(dev_lines)

In [74]:
evaluator = rouge.Rouge(['rouge-l'])

In [77]:
def evaluate(input_texts, target_texts, evaluator):
    predicted_texts = []
    for input_text in input_texts:
        input_seq = dataset.create_encoder_input_item_for_text(input_text)
        input_seq = np.array([input_seq])

        decoded_sentence = decode_sequence(input_seq)

        predicted_texts.append(decoded_sentence)

    return evaluator.get_scores(target_texts, predicted_texts, avg=True)

In [78]:
scores = evaluate(dev_input_texts[:10], dev_target_texts[:10], evaluator)

In [79]:
scores

{'rouge-l': {'f': 0.20906093478015747,
  'p': 0.2527380952380952,
  'r': 0.18571428571428567}}